# Split Chosen AoI Shapes into Grid Patches

The FloodMapper system requires area-of-interest (AoI) shapes that have been resampled by the fixed grid of processing 'patches' defined in the database. This notebook demonstrates how transform arbitrary AoI shapes into a GeoJSON file suitable for use with FloodMapper.

In [ ]:
# Necessary Python modules
import sys
import os
os.environ['USE_PYGEOS'] = '0'

import pandas as pd
import geopandas as gpd
import folium

from dotenv import load_dotenv
from db_utils import DB
from ml4floods.data import utils

# Set bucket will not be requester pays
utils.REQUESTER_PAYS_DEFAULT = False

## Load environment and project details

As with the other notebooks, we load credentials and project details from a hidden ```.env``` file.

In [ ]:
# Load environment variables (including path to credentials) from '.env' file
env_file_path = "../.env" 

# Uncomment for alternative version for Windows (r"" indicates raw string)
#env_file_path = r"C:/Users/User/floodmapper/.env"

assert load_dotenv(dotenv_path=env_file_path) == True, "[ERR] Failed to load environment!"
assert "GOOGLE_APPLICATION_CREDENTIALS" in os.environ, "[ERR] Missing $GOOGLE_APPLICATION_CREDENTIAL!"
assert "GS_USER_PROJECT" in os.environ, "[ERR] Missing $GS_USER_PROJECT!"
key_file_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
assert os.path.exists(key_file_path), f"[ERR] Google credential key file does not exist: \n{key_file_path} "
assert "ML4FLOODS_BASE_DIR" in os.environ, "[ERR] Missing $ML4FLOODS_BASE_DIR!"
base_path = os.environ["ML4FLOODS_BASE_DIR"]
assert os.path.exists(base_path), f"[ERR] Base path does not exist: \n{base_path} "
bucket_name = os.environ["BUCKET_URI"]
assert bucket_name is not None and bucket_name != "", f"Bucket name not defined {bucket_name}"

print("[INFO] Successfully loaded FloodMapper environment.")

In [ ]:
# Connect to the database (point to the .env file for credentials)
db_conn = DB(env_file_path)

## Select LGA shapes that overlap with the AoI


In [ ]:
# EDIT THIS CELL

# Set a session name
session_name = "EMSR570"

# Load the AoI shape that was drawn by hand
aoi_file = "EMSR570_AoI.gpkg"
aoi_initial_gdf = gpd.read_file(aoi_file)
aoi_initial_gdf.plot()

In [ ]:
# Fetch all the LGAs from the database
# TODO: make this a spatial query with the AoI
query = (f"SELECT lga_name22, ST_AsText(geometry_col) "
         f"FROM lgas_info;")
lga_df = db_conn.run_query(query, fetch=True)
print(f"[INFO] Returned {len(lga_df)} rows.")

# Format the results into a correct GeoDataFrame
lga_df['geometry'] = gpd.GeoSeries.from_wkt(lga_df['st_astext'])
lga_df.drop(['st_astext'], axis=1, inplace = True)
lga_gdf = gpd.GeoDataFrame(lga_df, geometry='geometry', crs="EPSG:4326")

In [ ]:
# Select the LGAs intersecting with the AoI
selected_lgas = lga_gdf.sjoin(aoi_initial_gdf, how="inner")
selected_lgas.drop(columns=["index_right", "id"], inplace=True)

# Drop odd Unincorporated NSW
selected_lgas.drop(selected_lgas[selected_lgas["lga_name22"] == "Unincorporated NSW"].index, inplace=True)

In [ ]:
selected_lgas

## Visualise the AoIs to be mapped

The ```selected_aois``` dataframe contains a 'geometry' column with the boundaries of the LGAs, which we can visualise to confirm that they correspond to our previous selection.

In [ ]:
# Plot the boundaries over OpenStreetMap layer
selected_lgas.explore()

## Load the pre-defined grid of processing patches

Next we load the pre-defined grid of square processing patches from the FloodMapper database. To do this we define a Strutured Query Language (SQL - 'sequel') query to fetch the information.

In [ ]:
# Define and execute the query
query = (f"SELECT patch_name, ST_AsText(geometry) "
         f"FROM grid_loc")
grid_df = db_conn.run_query(query, fetch= True)
print(f"[INFO] Returned {len(grid_df)} rows.")

# Format the results into a correct GeoDataFrame
grid_df['geometry'] = gpd.GeoSeries.from_wkt(grid_df['st_astext'])
grid_df.drop(['st_astext'], axis=1, inplace = True)
grid_gdf = gpd.GeoDataFrame(grid_df, geometry='geometry', crs="EPSG:4326")
grid_gdf.head(3)

## Use the AoIs to choose the grid patches to process

The next step is to filter the grid patches to be processed by choosing only those that overlap with the selected AoIs. 

In [ ]:
# Do a spatial join to choose grid patches that overlap with AoIs
valid_names = grid_gdf.sjoin(selected_lgas, how="inner")["patch_name"]
grid_selected_gdf = grid_gdf[grid_gdf.patch_name.isin(valid_names)]
grid_selected_gdf.drop_duplicates()
grid_selected_gdf.head(3)

In [ ]:
# Overplot the grid on the selected AOIs
m = selected_lgas.explore(style_kwds={"fillOpacity": 0.1}, name="Selected AoIs")
grid_selected_gdf.explore(m=m, style_kwds={"fillOpacity": 0.0}, color="red", name="Grid")
folium.LayerControl(collapsed=False).add_to(m)
m

## Save the gridded AoIs to the GCP bucket

The Floodmapper system (including the three main scripts) is driven by a saved version of these selected grid patches. The final step is to save the the selection to a GeoJSON format file on the GCP bucket.

In [ ]:
# Output filename
outfile = "patches_to_map.geojson"

# Form the session path and output path on the GCP bucket
session_path = os.path.join(bucket_name, "0_DEV/1_Staging/operational", 
                            session_name).replace("\\", "/")
outfile_path = os.path.join(session_path, outfile).replace("\\", "/")
print(f"[INFO] Will save selected grid patches to:\n\t{outfile_path}")

**Note down this path - the FloodMapper scripts will need to read this file on the command line.**

In [ ]:
# ML4Floods provides a utility to write to the GCP bucket
utils.write_geojson_to_gcp(outfile_path, grid_selected_gdf)

In [ ]:
# We can also check if the file was written successfully
fs = utils.get_filesystem(outfile_path)
success = fs.exists(outfile_path)
print(f"[INFO] Was file written successfully to GCP? -> {success}")

Finally, we can also write the selection to a GeoJSON file on our local disk.

In [ ]:
# Write the selected grid patches to local disk
folder_path = os.path.join(base_path, "flood-activations", session_name).replace("\\", "/")
outfile_path = os.path.join(folder_path, outfile).replace("\\", "/")
os.makedirs(folder_path, exist_ok=True)
grid_selected_gdf.to_file(outfile_path, driver="GeoJSON")
print(f"[INFO] Selected grid patches writen to:\n\t{outfile_path}")

In [ ]:
# Check that the file is on our local disk
!ls -l {folder_path}